In [1]:
import tensorflow as tf
import tensorflow_federated as tff
import Filter
import cv2
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Softmax
from tensorflow.keras.metrics import BinaryAccuracy, Precision, Recall
from Clients import Client

2023-02-08 23:48:53.142346: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-08 23:48:53.732836: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-02-08 23:48:53.732888: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-02-08 23:48:53.732894: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [2]:
emnist_train, emnist_test = tff.simulation.datasets.emnist.load_data(only_digits=True)

In [3]:
def preprocess(dataset):

  def batch_format_fn(element):
    """Flatten a batch of EMNIST data and return a (features, label) tuple."""
    return (tf.reshape(element['pixels'], [-1, 784]), 
            tf.reshape(element['label'], [-1, 1]))

  return dataset.batch(BATCH_SIZE).map(batch_format_fn)


In [4]:
#Initializes model to be transmitted to each client
def initialize_model(shape):
    model = Sequential([
        Dense(32, activation='relu', input_shape=shape),
        Dense(32, activation='relu'),
        Softmax()
    ])
    model.compile(optimizer='sgd',
                  loss='sparse_categorical_crossentropy',
                  metrics = ['accuracy'])
    return model

In [5]:
NUM_CLIENTS = 20
BATCH_SIZE = 32

clients = list()
client_ids = sorted(emnist_train.client_ids)[:NUM_CLIENTS]

for x in client_ids:
    clients.append(Client(x, preprocess(emnist_train.create_tf_dataset_for_client(x)),
                         initialize_model((784,))))
    
global_model = initialize_model((784,))

In [6]:
def get_global_count(clients):
    global_count = 0
    for c in clients:
        global_count += c.get_training_data_points()
    return global_count

In [7]:
def scale_weights(weight, scalar):
    f_weight = []
    for i in range(len(weight)):
        f_weight.append(scalar * weight[i])
    return f_weight

In [8]:
def sum_scaled_weights(weight_lst):
    avg = list()
    for lst_tuple in zip(*weight_lst):
        layer_mean = tf.math.reduce_sum(lst_tuple, axis=0)
        avg.append(layer_mean)
    return avg

In [9]:
def prep_test_data(data):
    data_pixels, data_labels = list(), list()
    for d in data:
        data_pixels.append(tf.reshape(d['pixels'], [-1, 784]))
        data_labels.append(str(tf.reshape(d['label'], [-1, 1]).numpy()[0][0]))
    return np.array(data_pixels), np.array(data_labels)

In [10]:
emnist_test = emnist_test.create_tf_dataset_from_all_clients()
test_data = preprocess(emnist_test)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [11]:
comm_rounds = 20
for r in range(comm_rounds):
    print('-'*50)
    print('[INFO] Communication round {}'.format(r+1))
    global_weights = global_model.get_weights() #get global model weights
    scaled_weights = list()
    
    for c in range(len(clients)):
        clients[c].train(global_weights)
        
        #Get scaling factor for local weights
        bs = clients[c].get_batch_size()
        gc = get_global_count(clients) * bs
        lc = clients[c].get_training_data_points() * bs
        sf = lc / gc
        #Scale local weights with scalar
        scaled_weights.append(scale_weights(clients[c].model.get_weights(), sf))
        
    fed_avg = sum_scaled_weights(scaled_weights)
    #Update global model with avg weights
    global_model.set_weights(fed_avg)
    
    #Eval global model
    print("EVALUATING...")
    result = global_model.evaluate(test_data)
    print("EVALUATION COMPLETE...")
    

--------------------------------------------------
[INFO] Communication round 1
      1/Unknown - 1s 727ms/step - loss: 3.7403 - accuracy: 0.0312

2023-02-08 23:48:57.382625: W tensorflow/compiler/xla/service/gpu/nvptx_helper.cc:56] Can't find libdevice directory ${CUDA_DIR}/nvvm/libdevice. This may result in compilation or runtime failures, if the program we try to run uses routines from libdevice.
Searched for CUDA in the following directories:
  ./cuda_sdk_lib
  /usr/local/cuda-11.2
  /usr/local/cuda
  .
You can choose the search directory by setting xla_gpu_cuda_data_dir in HloModule's DebugOptions.  For most apps, setting the environment variable XLA_FLAGS=--xla_gpu_cuda_data_dir=/path/to/cuda will work.


3/3 [==============================] - 0s 10ms/step - loss: 3.4110 - accuracy: 0.0645


3/3 [==============================] - 0s 9ms/step - loss: 3.3801 - accuracy: 0.0694
EVALUATING...
1276/1276 [==============================] - 2s 1ms/step - loss: 3.2639 - accuracy: 0.0874
EVALUATION COMPLETE...
--------------------------------------------------
[INFO] Communication round 2
3/3 [==============================] - 0s 9ms/step - loss: 3.2766 - accuracy: 0.0972
EVALUATING...
1276/1276 [==============================] - 2s 1ms/step - loss: 3.1967 - accuracy: 0.1007
EVALUATION COMPLETE...
--------------------------------------------------
[INFO] Communication round 3
3/3 [==============================] - 0s 9ms/step - loss: 3.2334 - accuracy: 0.1250
EVALUATING...
1276/1276 [==============================] - 2s 1ms/step - loss: 3.1590 - accuracy: 0.1036
EVALUATION COMPLETE...
--------------------------------------------------
[INFO] Communication round 4
3/3 [==============================] - 0s 9ms/step - loss: 3.2117 - accuracy: 0.1389
EVALUATING...
1276/1276 [===========

3/3 [==============================] - 0s 10ms/step - loss: 3.1910 - accuracy: 0.0694
EVALUATING...
1276/1276 [==============================] - 2s 1ms/step - loss: 3.0958 - accuracy: 0.0982
EVALUATION COMPLETE...
--------------------------------------------------
[INFO] Communication round 7
3/3 [==============================] - 0s 8ms/step - loss: 3.1692 - accuracy: 0.0694
EVALUATING...
1276/1276 [==============================] - 2s 1ms/step - loss: 3.0625 - accuracy: 0.0982
EVALUATION COMPLETE...
--------------------------------------------------
[INFO] Communication round 8
3/3 [==============================] - 0s 9ms/step - loss: 3.1509 - accuracy: 0.0694
EVALUATING...
1276/1276 [==============================] - 2s 1ms/step - loss: 3.0411 - accuracy: 0.0987
EVALUATION COMPLETE...
--------------------------------------------------
[INFO] Communication round 9
3/3 [==============================] - 0s 10ms/step - loss: 3.1376 - accuracy: 0.0833
EVALUATING...
   1072/Unknown - 1s

KeyboardInterrupt: 